# Import Libraries

In [2]:
import re
from collections import defaultdict


# Parse Conversation Function

In [3]:
def parse_conversation(conversation_text):
    """
    Parses the conversation into dialogues with metadata.

    Args:
        conversation_text (str): The conversation text.

    Returns:
        list: A list of dictionaries containing dialogue number, role, and dialogue text.
    """
    dialogue_list = []
    # Adjusted role patterns to accurately capture roles
    role_patterns = {
        'Recovery Agent': r'RA:',
        'Borrower': r'B:'
    }
    # Split the conversation text based on role identifiers
    potential_dialogues = re.split(r'(RA:|B:)', conversation_text.strip())
    dialogue_number = 1
    i = 1
    while i < len(potential_dialogues):
        role_indicator = potential_dialogues[i]
        text = potential_dialogues[i + 1].strip()
        role = 'Recovery Agent' if role_indicator == 'RA:' else 'Borrower'
        dialogue_data = {
            'dialogue_number': dialogue_number,
            'role': role,
            'dialogue': text,
            'themes': [],
            'sentiments': []
        }
        dialogue_list.append(dialogue_data)
        dialogue_number += 1
        i += 2  # Move to the next role indicator
    return dialogue_list


# Tokenize Text Function

In [4]:
def tokenize_text(text):
    """
    Tokenizes the text into words.

    Args:
        text (str): The text to tokenize.

    Returns:
        list: A list of tokens.
    """
    tokens = re.findall(r'\b\w+\b', text)
    return tokens


# Analyze Dialogues Function



In [5]:
def analyze_dialogues(dialogues):
    """
    Performs thematic and sentiment analysis on the dialogues.

    Args:
        dialogues (list): List of dialogues.

    Returns:
        list: Updated list of dialogues with themes and sentiments.
    """
    themes = {
        'Financial Difficulty': ['नौकरी', 'पैसा', 'समस्या', 'दिक्कत', 'मुश्किल', 'आर्थिक', 'चुनौती', 'पैसे नहीं'],
        'Payment Issue': ['EMI', 'भुगतान', 'लोन', 'किस्त', 'बकाया', 'देय', 'बकाया'],
        'Negotiation': ['समय', 'प्लान', 'कोशिश', 'समझौता', 'प्रस्ताव', 'बातचीत', 'आधा', 'राहत', 'देरी'],
        'Empathy': ['बुरा हुआ', 'दुर्भाग्य', 'अफसोस', 'समझते हैं', 'सहानुभूति', 'दुख'],
        'Professional Conduct': ['जरूरी', 'महत्वपूर्ण', 'पालन', 'समय पर', 'नियम', 'शर्तें', 'अनुशासन', 'समझना होगा']
    }

    sentiments = {
        'Empathetic': ['बुरा हुआ', 'समझते हैं', 'दुख', 'अफसोस'],
        'Firm': ['समझना होगा', 'जरूरी है', 'समय पर करना', 'बहुत जरूरी है'],
        'Distressed': ['दिक्कत', 'पैसे नहीं', 'मुश्किल', 'नौकरी चली गई'],
        'Hopeful': ['उम्मीद', 'कोशिश करूंगा', 'मिल जाएगा'],
        'Cooperative': ['ठीक रहेगा', 'पालन करूंगा', 'धन्यवाद', 'करूंगा']
    }

    for dialogue in dialogues:
        text = dialogue['dialogue']
        tokens = tokenize_text(text)
        # Thematic analysis
        for theme, keywords in themes.items():
            if any(keyword in text for keyword in keywords):
                dialogue['themes'].append(theme)
        # Sentiment analysis
        for sentiment, keywords in sentiments.items():
            if any(keyword in text for keyword in keywords):
                dialogue['sentiments'].append(sentiment)
    return dialogues


# Extract Insights Function

In [6]:
def extract_insights(dialogues):
    """
    Extracts key actions, communication styles, and negotiation strategies.

    Args:
        dialogues (list): List of dialogues with themes and sentiments.

    Returns:
        dict: Insights including actions, communication styles, and negotiation strategies.
    """
    insights = {
        'actions': defaultdict(set),
        'communication_styles': {},
        'negotiation_strategies': {}
    }

    for dialogue in dialogues:
        role = dialogue['role']
        text = dialogue['dialogue']
        themes = dialogue['themes']
        sentiments = dialogue['sentiments']

        # Extract actions
        if 'Payment Issue' in themes and role == 'Recovery Agent':
            if 'SMS' in text or 'संदेश' in text:
                insights['actions'][role].add('Send SMS with payment details.')
            if 'भुगतान' in text and ('करें' in text or 'करेगा' in text):
                insights['actions'][role].add('Request payment from the borrower.')
        if 'Negotiation' in themes and role == 'Borrower':
            insights['actions'][role].add('Propose a modified payment plan.')
        if 'Payment Issue' in themes and role == 'Borrower':
            insights['actions'][role].add('Agree to make payments as per the new plan.')

        # Communication styles
        if role not in insights['communication_styles']:
            if 'Empathetic' in sentiments:
                insights['communication_styles'][role] = 'Uses empathetic communication.'
            elif 'Firm' in sentiments:
                insights['communication_styles'][role] = 'Maintains a firm tone.'
            else:
                insights['communication_styles'][role] = 'Communicates openly.'

        # Negotiation strategies
        if role not in insights['negotiation_strategies']:
            if 'Negotiation' in themes:
                insights['negotiation_strategies'][role] = 'Engages in negotiation.'
            elif 'Firm' in sentiments:
                insights['negotiation_strategies'][role] = 'Sets clear expectations.'
            else:
                insights['negotiation_strategies'][role] = 'Standard communication.'

    # Convert action sets to lists
    for role in insights['actions']:
        insights['actions'][role] = list(insights['actions'][role])

    return insights


# Generate Report Function

In [7]:
def generate_report(dialogues, insights):
    """
    Generates and prints the analysis report.

    Args:
        dialogues (list): List of dialogues with themes and sentiments.
        insights (dict): Insights extracted from the dialogues.
    """
    summary_sentences = set()
    for dialogue in dialogues:
        role = dialogue['role']
        themes = dialogue['themes']
        if 'Financial Difficulty' in themes and role == 'Borrower':
            summary_sentences.add('The borrower explains financial difficulties due to job loss.')
        if 'Payment Issue' in themes and role == 'Recovery Agent':
            summary_sentences.add('The recovery agent discusses the missed EMI payment.')
        if 'Negotiation' in themes:
            summary_sentences.add('Negotiation occurs regarding payment amounts and deadlines.')
        if 'Empathy' in themes and role == 'Recovery Agent':
            summary_sentences.add('The agent shows empathy towards the borrower.')

    summary = '. '.join(summary_sentences) + '.'
    summary_words = summary.split()
    if len(summary_words) > 100:
        summary = ' '.join(summary_words[:100]) + '.'

    # Key actions
    key_actions = []
    for role, actions_list in insights['actions'].items():
        for action in actions_list:
            key_actions.append(f"{role}: {action}")
    key_actions = key_actions[:5]

    # Sentiment Analysis
    agent_sentiments = ', '.join(set(sent for d in dialogues if d['role'] == 'Recovery Agent' for sent in d['sentiments']))
    borrower_sentiments = ', '.join(set(sent for d in dialogues if d['role'] == 'Borrower' for sent in d['sentiments']))

    agent_tone = f"The recovery agent's tone is {agent_sentiments}."
    borrower_tone = f"The borrower's tone is {borrower_sentiments}."

    # Additional insights
    communication_styles = insights['communication_styles']
    negotiation_strategies = insights['negotiation_strategies']

    # Print the report
    print("### 1. Summary of the Conversation\n")
    print(summary)
    print(f"\nWord count: {len(summary_words)} words")
    print("\n### 2. Key Actions and Next Steps\n")
    for idx, action in enumerate(key_actions, 1):
        print(f"{idx}. {action}")
    print("\n### 3. Sentiment Analysis\n")
    print("**Recovery Agent's Tone:**")
    print(agent_tone)
    print("\n**Borrower's Tone:**")
    print(borrower_tone)
    print("\n### 4. Additional Insights\n")
    print("#### Communication Styles")
    for role, style in communication_styles.items():
        print(f"- **{role}:** {style}")
    print("\n#### Negotiation Strategies")
    for role, strategy in negotiation_strategies.items():
        print(f"- **{role}:** {strategy}")
    print()


# Main Function

In [8]:
def main():
    """
    Main function to execute the analysis.
    """
    # Input the conversation text
    print("Please paste the conversation text below (end input with an empty line):\n")
    conversation_lines = []
    while True:
        try:
            line = input()
            if line.strip() == '':
                break
            conversation_lines.append(line)
        except EOFError:
            break

    conversation_text = ' '.join(conversation_lines)

    dialogues = parse_conversation(conversation_text)
    dialogues = analyze_dialogues(dialogues)
    insights = extract_insights(dialogues)
    generate_report(dialogues, insights)

if __name__ == "__main__":
    main()


Please paste the conversation text below (end input with an empty line):

RA: नमस्ते मिस्टर गुप्ता, मैं पीक्यूआर फाइनेंस से बोल रहा हूं। आपके लोन के बारे में कुछ चर्चा करनी थी। B: हाँ, बताइए। क्या समस्या है? RA: आपकी पिछली EMI इस महीने नहीं आई है। क्या कोई दिक्कत है? B: जी, मेरी दुकान में आग लग गई थी, इसलिए व्यापार में नुकसान हुआ है। RA: ओह, यह सुनकर बहुत अफसोस हुआ। क्या आप हमें बता सकते हैं कि आप कब तक भुगतान कर पाएंगे? B: मैं अभी स्थिति संभालने की कोशिश कर रहा हूं। क्या मुझे एक महीने का समय मिल सकता है? RA: हम आपकी स्थिति समझते हैं। क्या आप अगले महीने तक भुगतान कर पाएंगे? B: जी हाँ, मैं अगले महीने तक पूरी राशि चुका दूंगा। RA: ठीक है, मैं आपके खाते में नोट कर देता हूँ। कृपया समय पर भुगतान करें। B: धन्यवाद आपकी समझदारी के लिए। RA: कोई बात नहीं। अगर आपको और सहायता चाहिए तो हमें बताएं। B: जरूर, धन्यवाद।

### 1. Summary of the Conversation

The borrower explains financial difficulties due to job loss.. The agent shows empathy towards the borrower.. Negotiation occurs regarding payment amo